# 02 — Preprocessamento de Texto

Demonstração e aplicação do pipeline de preprocessamento sobre o dataset B2W.

**Etapas:**
1. Limpeza de texto (lowercase, URLs, pontuação, números)
2. Remoção de stopwords (NLTK PT)
3. Stemming (RSLP)
4. Lematização (spaCy — demonstrativo)
5. Aplicação ao dataset completo

In [1]:
import sys
sys.path.insert(0, "..")

import pandas as pd
from tqdm.auto import tqdm

from src.config import TEXT_COLUMN, PROCESSED_DIR
from src.data_loader import carregar_splits
from src.preprocessing import (
    limpar_texto,
    remover_stopwords,
    aplicar_stemming,
    aplicar_lematizacao,
    preprocessar_para_svm,
    preprocessar_para_embeddings,
)
from src.utils import set_seed, timer

set_seed()
tqdm.pandas()

## 1. Carregar Splits

In [2]:
df_train, df_val, df_test = carregar_splits()
df_train.head(3)

Splits carregados: treino=90,368 | val=19,365 | teste=19,365


,submission_date,reviewer_id,product_id,product_name,product_brand,site_category_lv1,site_category_lv2,review_title,overall_rating,recommend_to_a_friend,review_text,reviewer_birth_year,reviewer_gender,reviewer_state,sentimento,label,num_chars,num_words
0,2018-01-01 09:59:57,1e926b3472c7e45c4102eafdce65be07f7dac5d0105c7d...,27798113,"Quarto Infantil com Guarda Roupa 3 Portas, Côm...",None,Móveis,Quarto Completo,None,5,Yes,"Gostei muito dos produtos, guarda roupa grande...",1968.0,F,RJ,positivo,2,295,50
1,2018-01-16 06:51:53,bb764dae5ddf9f5a795715976d077eb1222919f93a278e...,14171433,Lanterna De Cabeca Tatica Cree Led Com Foco Aj...,None,Esporte e Lazer,Camping,Nao e o que eu esperava,2,No,A propaganda informava alcance de ate 250mts. ...,1947.0,M,SP,negativo,0,72,13
2,2018-04-26 07:39:08,d11bd8ec2271d0f374c5137502ec041a0701f5ee2fbad3...,132918761,Conjunto de Refratários Marinex - 10 Peças,None,Utilidades Domésticas,"Bandejas, Assadeiras e Réchaud",Pratico:,5,Yes,"Eu amei , era exatamente o que eu esperava , ...",1967.0,F,GO,positivo,2,67,13


## 2. Demonstração das Etapas (Antes/Depois)

In [3]:
# Selecionar exemplos representativos
exemplos = df_train.groupby("sentimento").apply(
    lambda x: x.sample(1, random_state=42)
).reset_index(drop=True)

for _, row in exemplos.iterrows():
    texto = row[TEXT_COLUMN]
    print(f"=== {row['sentimento'].upper()} ===")
    print(f"Original:     {texto[:200]}..." if len(str(texto)) > 200 else f"Original:     {texto}")
    print(f"Limpo:        {limpar_texto(texto)[:200]}")
    print(f"Sem stopwords:{remover_stopwords(limpar_texto(texto))[:200]}")
    print(f"Com stemming: {aplicar_stemming(remover_stopwords(limpar_texto(texto)))[:200]}")
    print(f"Pipeline SVM: {preprocessar_para_svm(texto)[:200]}")
    print()

=== NEGATIVO ===
Original:     Fonte minuscula não dá para ler, a não ser que você use um telescópio, esperdicei meu dinheiro.
Limpo:        fonte minuscula não dá para ler a não ser que você use um telescópio esperdicei meu dinheiro
Sem stopwords:fonte minuscula dá ler use telescópio esperdicei dinheiro
Com stemming: font minuscul dá ler use telescópi esperdic dinh
Pipeline SVM: font minuscul dá ler use telescópi esperdic dinh

=== NEUTRO ===
Original:     gostei, mas achei um pouco pequena, poderia ser maior, a tapióca fica pequena kk mais fica uma delícia.
Limpo:        gostei mas achei um pouco pequena poderia ser maior a tapióca fica pequena kk mais fica uma delícia
Sem stopwords:gostei achei pouco pequena poderia maior tapióca fica pequena kk fica delícia
Com stemming: gost ach pouc pequen pod mai tapióc fic pequen kk fic delíc
Pipeline SVM: gost ach pouc pequen pod mai tapióc fic pequen kk fic delíc

=== POSITIVO ===
Original:     O produto é excelente! Além de filtrar a água pa

/var/folders/0x/f3ysthvx15g71kxkdntwphqr0000gq/T/ipykernel_18381/125629128.py:2: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  exemplos = df_train.groupby("sentimento").apply(


### 2.1 Lematização (demonstrativo)
Mais lenta que stemming — usada apenas para referência.

In [4]:
exemplo = exemplos.iloc[0][TEXT_COLUMN]
texto_limpo = remover_stopwords(limpar_texto(exemplo))
print(f"Stemming:     {aplicar_stemming(texto_limpo)[:200]}")
print(f"Lematização:  {aplicar_lematizacao(texto_limpo)[:200]}")

Stemming:     font minuscul dá ler use telescópi esperdic dinh


Lematização:  fonte minuscular dar ler usar telescópio esperdicei dinheiro


## 3. Aplicação ao Dataset Completo

Criamos duas colunas preprocessadas:
- `texto_svm`: para TF-IDF + SVM (com stemming)
- `texto_emb`: para embeddings (sem stemming)

In [5]:
def preprocessar_split(df: pd.DataFrame, nome: str) -> pd.DataFrame:
    """Aplica preprocessamento a um split."""
    print(f"Preprocessando {nome} ({len(df):,} amostras)...")
    with timer(f"Pipeline SVM — {nome}"):
        df["texto_svm"] = df[TEXT_COLUMN].progress_apply(preprocessar_para_svm)
    with timer(f"Pipeline Embeddings — {nome}"):
        df["texto_emb"] = df[TEXT_COLUMN].progress_apply(preprocessar_para_embeddings)
    return df

In [6]:
df_train = preprocessar_split(df_train, "treino")
df_val = preprocessar_split(df_val, "validação")
df_test = preprocessar_split(df_test, "teste")

Preprocessando treino (90,368 amostras)...


  0%|          | 0/90368 [00:00<?, ?it/s]

⏱ Pipeline SVM — treino: 27.4s


  0%|          | 0/90368 [00:00<?, ?it/s]

⏱ Pipeline Embeddings — treino: 2.1s
Preprocessando validação (19,365 amostras)...


...

⏱ Pipeline Embeddings — validação: 0.5s
Preprocessando teste (19,365 amostras)...


  0%|          | 0/19365 [00:00<?, ?it/s]

⏱ Pipeline SVM — teste: 5.8s


  0%|          | 0/19365 [00:00<?, ?it/s]

⏱ Pipeline Embeddings — teste: 0.5s


In [7]:
# Verificar resultados
print("\nExemplos preprocessados:")
df_train[[TEXT_COLUMN, "texto_svm", "texto_emb", "sentimento"]].head(5)


Exemplos preprocessados:


,review_text,texto_svm,texto_emb,sentimento
0,"Gostei muito dos produtos, guarda roupa grande...",gost produt guard roup grand espaç comod grand...,gostei produtos guarda roupa grande espaçoso c...,positivo
1,A propaganda informava alcance de ate 250mts. ...,propagand inform alcanc ate mt por nao cheg at...,propaganda informava alcance ate mts porem nao...,negativo
2,"Eu amei , era exatamente o que eu esperava , ...",ame exat esper gost recom,amei exatamente esperava gostei recomendo,positivo
3,"um pouco mais frágil do que parece na foto, ve...",pouc frágil parec fot vei falt um parafus mont...,pouco frágil parece foto veio faltando uns par...,negativo
4,"Tablet trava o tempo inteiro, não aceita insta...",tablet tr temp int aceit instal jog registr vá...,tablet trava tempo inteiro aceita instalação j...,negativo


## 4. Salvar Splits Preprocessados

In [8]:
df_train.to_parquet(PROCESSED_DIR / "train.parquet", index=False)
df_val.to_parquet(PROCESSED_DIR / "val.parquet", index=False)
df_test.to_parquet(PROCESSED_DIR / "test.parquet", index=False)
print("Splits preprocessados salvos com sucesso!")

Splits preprocessados salvos com sucesso!


## Resumo

- Pipeline de limpeza aplicada: lowercase, remoção de URLs/números/pontuação
- Stopwords PT removidas (NLTK)
- Stemming RSLP aplicado para SVM
- Colunas `texto_svm` e `texto_emb` criadas nos splits
- Splits atualizados em `data/processed/`
- Próximo passo: **03_svm_bow.ipynb**